**Bidirectional Encoder Representations from Transformers (BERT)**

Это файл о моем первом опыте с BERT
<br><br>

Статьи, которые помогли разбраться:
<br>

[Визуализируя нейронный машинный перевод (seq2seq модели с механизмом внимания)](https://habr.com/ru/post/486158/)

[Transformer в картинках](https://habr.com/ru/post/486358/)

[BERT, ELMO и Ко в картинках (как в NLP пришло трансферное обучение)](https://habr.com/ru/post/487358/)

[Ваш первый BERT: иллюстрированное руководство](https://habr.com/ru/post/498144/)

[A Visual Notebook to Using BERT for the First TIme.ipynb](https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb)
<br><br>
Код в целом без пояснений и комментариев, так как все подробно изложен в приведенных статьях.
<br><br>





In [ ]:
# тестовый запуск, проверка работоспособности блокнота
print('test')

test


In [ ]:
# устанавливаем трансформеры
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 5.1 MB/s eta 0:00:00


In [ ]:
# устанавливаем библиотеки

import numpy as np
import pandas as pd
import torch
import transformers as ppb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# скачиваем dataset
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [ ]:
# выбираем фрагмент для ускорения и экономии времени
batch_1 = df[:2000]

In [ ]:
# проверяем, что с данными все ок
print(batch_1[:5])

                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
3  this is a visually stunning rumination on love...  1
4  jonathan parker 's bartleby should have been t...  1


In [ ]:
# Загружаем предобученную модель и токенизаторы 
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# токенизируем 
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
# делаем из списков массив, чтобы была одинаковая длина

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
# маскируем сделанные добавления
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
# создаем входной вектор из матрицы токенов

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# разрезаем этот 3d тензор, чтобы получить нужный нам 2d тензор
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
# разделяем данные на обучающую и тестовую выборки
labels = batch_1[1]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# обучаем модель логистической регрессии на обучающей выборке
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
# модель обучена, смотрим метрики 
print('# train:',lr_clf.score(train_features, train_labels))
print('# test:',lr_clf.score(test_features, test_labels))

# train: 0.906
# test: 0.844



**Собственная часть**

Проверим, что все корректно

In [ ]:
# распечатаем фрагмент из датасета
print(batch_1[:5])

                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
3  this is a visually stunning rumination on love...  1
4  jonathan parker 's bartleby should have been t...  1


In [ ]:
# посчитаем сами и сравним

def LR(x): return (1 if x > 0 else 0)
def correct(LR, label): return True if LR - label == 0 else False

sum_false = 0
N = 5

for i in range (N):
  if i and i % 100 == 0: print(i)
  array = np.array(tokenized[i])
  array = np.pad(array, (0, max_len - len(array)), 'constant')
  attention_mask_this = np.where(array != 0, 1, 0)
  input_ids_this = torch.tensor([array])  
  attention_mask_this = torch.tensor(attention_mask_this)
  with torch.no_grad():
      last_hidden_states_this = model(input_ids_this, attention_mask=attention_mask_this)
  features_this = last_hidden_states_this[0][:,0,:].numpy()

  sum = np.dot(features_this,lr_clf.coef_[0]) 

  if N < 20: print(i, labels[i], LR(sum), correct(LR(sum), labels[i]), sum)  
  if correct(LR(sum), labels[i]) == True: sum_false += 1

print()
print(sum_false/(i+1))  

0 1 1 True [3.85857511]
1 0 0 True [-6.51904703]
2 0 0 True [-2.21694384]
3 1 1 True [3.65850942]
4 1 0 False [-0.32528463]

0.8


Работаем со своими фрагментами текста
<br><br>

In [ ]:
# задаем свои фрагменты

texts = [
    'All is good', 
    'it is so bad', 
    'nice to meet you',
    'it is so rainy',
    'he is a stupid',
    'I like my car',
    'have a nice day'    
    ]

In [ ]:
# Классифицируем свои фрагменты

for text in texts:
  array = tokenizer.encode(text, add_special_tokens=True)
  array = np.pad(array, (0, max_len - len(array)), 'constant')
  attention_mask_this = np.where(array != 0, 1, 0)
  input_ids_this = torch.tensor([array])  
  attention_mask_this = torch.tensor(attention_mask_this)
  with torch.no_grad():
      last_hidden_states_this = model(input_ids_this, attention_mask=attention_mask_this)
  features_this = last_hidden_states_this[0][:,0,:].numpy()

  sum = np.dot(features_this,lr_clf.coef_[0]) 

  print(text, ':', sum, ':', LR(sum)) 

All is good : [1.90097556] : 1
it is so bad : [-1.86999172] : 0
nice to meet you : [4.96587936] : 1
it is so rainy : [-2.53429642] : 0
he is a stupid : [-2.4318853] : 0
I like my car : [0.2118134] : 1
have a nice day : [2.30864912] : 1


In [ ]:
# На этом все )